# <center> Train LSTM with word2vec embeddings </center>

First I would like to mention these two excellent posts: 

https://www.kaggle.com/lystdo/lstm-with-word2vec-embeddings
https://blog.keras.io/using-pre-trained-word-embeddings-in-a-keras-model.html

My solution is in part based on these two guidelines.

Okay now let's import all of the necessary modules:

In [1]:
import os
import gensim
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from unidecode import unidecode
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.models import load_model
from sklearn.metrics import log_loss, accuracy_score
from sklearn.metrics import roc_curve, auc
from capstone_project import preprocessor as pre
from capstone_project.models import neural_nets

%matplotlib inline

Using TensorFlow backend.


Set important constants and load data:

In [2]:
MAX_SEQUENCE_LENGTH = 50 # Maximum length of input for lstm the maximum number of tokens is 103 
EMBEDDING_DIM = 300  # Length of the used word2vec implementation

In [3]:
file_directory = "../output/data/"
prefix = "tokenized_"

train_data = pre.load_pickle(file_directory, prefix+"train_data.pkl")
val_data = pre.load_pickle(file_directory, prefix+"val_data.pkl")  # Validation data set used to compare different classification algorithms
train_y = train_data["is_duplicate"].values
val_y = val_data["is_duplicate"].values

In [4]:
#train_data = train_data[:100]
#val_data = val_data[:100]

Prepare the tokenized question as input for keras:

In [5]:
# Decode again and join strings because keras tokenizer crashes when using unicode while spacy uses it
q1_tokens = train_data["q1_tokens"].apply(lambda x: unidecode(u" ".join(x))).values
q2_tokens = train_data["q2_tokens"].apply(lambda x: unidecode(u" ".join(x))).values
all_tokens = np.concatenate([q1_tokens, q2_tokens])

tokenizer = Tokenizer()
tokenizer.fit_on_texts(all_tokens)

word_index = tokenizer.word_index
number_words = len(word_index)+1  # Needed for embedding layer
print("Found {} unique tokens".format(len(word_index)))

q1_sequences = tokenizer.texts_to_sequences(q1_tokens)
q2_sequences = tokenizer.texts_to_sequences(q2_tokens)

q1_data = pad_sequences(q1_sequences, maxlen=MAX_SEQUENCE_LENGTH)
q2_data = pad_sequences(q2_sequences, maxlen=MAX_SEQUENCE_LENGTH)

Found 66580 unique tokens


split second val and train set for validation at every epoch:

In [6]:
# Split the data into another training set and a second validation set 
# See: https://blog.keras.io/using-pre-trained-word-embeddings-in-a-keras-model.html and
indices = np.arange(q1_data.shape[0])
np.random.shuffle(indices)
q1_data = q1_data[indices]
q2_data = q2_data[indices]
labels = train_y[indices]

nb_validation_samples = int(0.1 * q1_data.shape[0])

# Create subset for training with early stopping
q1_train = q1_data[:-nb_validation_samples]
q2_train = q1_data[:-nb_validation_samples]
train_labels = labels[:-nb_validation_samples]

#create validation subset that is used to validate each epoch during training
q1_val_epochs = q1_data[-nb_validation_samples:]
q2_val_epochs = q2_data[-nb_validation_samples:]
val_epochs_labels = labels[-nb_validation_samples:]

Doble the dataset size by switching the order of the questions. This is done in order to avoid symmetry issues.

In [7]:
# kaggle ...
q12_data = np.concatenate((q1_data, q2_data), axis=0)
q21_data = np.concatenate((q2_data, q1_data), axis=0)
double_labels = np.concatenate((labels, labels), axis=0)

q12_train = np.concatenate((q1_train, q2_train), axis=0)
q21_train = np.concatenate((q2_train, q1_train), axis=0)
double_train_labels = np.concatenate((train_labels, train_labels), axis=0)

q12_val_epochs = np.concatenate((q1_val_epochs, q2_val_epochs), axis=0)
q21_val_epochs = np.concatenate((q2_val_epochs, q1_val_epochs), axis=0)
double_val_epochs_labels = np.concatenate((val_epochs_labels, val_epochs_labels), axis=0)

Prepare validation set:

In [8]:
#create correct embeddings for validation data
q1_val_tokens = val_data["q1_tokens"].apply(lambda x: unidecode(u" ".join(x))).values
q2_val_tokens = val_data["q2_tokens"].apply(lambda x: unidecode(u" ".join(x))).values

q1_val_sequences = tokenizer.texts_to_sequences(q1_val_tokens)
q2_val_sequences = tokenizer.texts_to_sequences(q2_val_tokens)

q1_val_data = pad_sequences(q1_val_sequences, maxlen=MAX_SEQUENCE_LENGTH)
q2_val_data = pad_sequences(q2_val_sequences, maxlen=MAX_SEQUENCE_LENGTH)
val_labels = val_y

Load the pretrained word2vec model:

In [9]:
word2vec_model = gensim.models.KeyedVectors.load_word2vec_format("../data/GoogleNews-vectors-negative300.bin.gz", binary=True)

Set the parameters of the lstm, create the emebedding matrix and create a keras model:

In [15]:
batch_size = 2048
nn_parameters = {"max_sequence_length": MAX_SEQUENCE_LENGTH,
                 "num_lstm": 256,
                 "dropout_lstm": 0.5,
                 "num_dense": [256, 128],
                 "dropout_dense": 0.5}

stamp = "{}_{:.2f}_{}_{:.2f}".format(nn_parameters["num_lstm"], 
                                    nn_parameters["dropout_lstm"],
                                    nn_parameters["num_dense"],
                                    nn_parameters["dropout_dense"])

embedding_matrix = neural_nets.create_embedding_matrix(word2vec_model, EMBEDDING_DIM, word_index, number_words)

model = neural_nets.create_lstm(embedding_matrix, EMBEDDING_DIM, number_words, **nn_parameters)
model.compile(loss='binary_crossentropy', optimizer='nadam', metrics=['acc'])
model.summary()
print stamp

Null word embeddings: 30612
____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_5 (InputLayer)             (None, 50)            0                                            
____________________________________________________________________________________________________
input_6 (InputLayer)             (None, 50)            0                                            
____________________________________________________________________________________________________
embedding_3 (Embedding)          (None, 50, 300)       19974300    input_5[0][0]                    
                                                                   input_6[0][0]                    
____________________________________________________________________________________________________
lstm_3 (LSTM)                    (None, 256)           570368  

Train the data and check the performance on the second validation set every epoch. with early stopping:

In [16]:
# See https://www.kaggle.com/lystdo/lstm-with-word2vec-embeddings


early_stopping = EarlyStopping(monitor='val_loss', patience=50)
best_model_path = "../output/models/lstm_val_epochs_" + stamp + '.h5'
model_checkpoint = ModelCheckpoint(best_model_path, save_best_only=True, save_weights_only=True)

hist = model.fit([q12_train, q21_train], double_train_labels,
                 validation_data=([q12_val_epochs, q21_val_epochs], double_val_epochs_labels), 
                 epochs=300, batch_size=batch_size, shuffle=True,
                 callbacks=[early_stopping, model_checkpoint])

Train on 524038 samples, validate on 58226 samples
Epoch 1/300
524038/524038 [==============================] - 324s - loss: 0.6467 - acc: 0.6473 - val_loss: 0.6126 - val_acc: 0.6389
Epoch 2/300
524038/524038 [==============================] - 323s - loss: 0.5868 - acc: 0.6894 - val_loss: 0.5639 - val_acc: 0.7141
Epoch 3/300
524038/524038 [==============================] - 323s - loss: 0.5651 - acc: 0.7081 - val_loss: 0.5388 - val_acc: 0.7270
Epoch 4/300
524038/524038 [==============================] - 323s - loss: 0.5507 - acc: 0.7197 - val_loss: 0.5274 - val_acc: 0.7372
Epoch 5/300
524038/524038 [==============================] - 323s - loss: 0.5403 - acc: 0.7273 - val_loss: 0.5186 - val_acc: 0.7438
Epoch 6/300
524038/524038 [==============================] - 323s - loss: 0.5314 - acc: 0.7331 - val_loss: 0.5141 - val_acc: 0.7460
Epoch 7/300
524038/524038 [==============================] - 323s - loss: 0.5248 - acc: 0.7380 - val_loss: 0.5090 - val_acc: 0.7489
Epoch 8/300
524038/524038

524038/524038 [==============================] - 324s - loss: 0.5523 - acc: 0.7149 - val_loss: 0.5354 - val_acc: 0.7321
Epoch 63/300
524038/524038 [==============================] - 324s - loss: 0.5514 - acc: 0.7157 - val_loss: 0.5258 - val_acc: 0.7389
Epoch 64/300
524038/524038 [==============================] - 324s - loss: 0.5495 - acc: 0.7172 - val_loss: 0.5246 - val_acc: 0.7376
Epoch 65/300
524038/524038 [==============================] - 324s - loss: 0.5474 - acc: 0.7187 - val_loss: 0.5245 - val_acc: 0.7391
Epoch 66/300
524038/524038 [==============================] - 324s - loss: 0.5470 - acc: 0.7192 - val_loss: 0.5286 - val_acc: 0.7364
Epoch 67/300
524038/524038 [==============================] - 324s - loss: 0.5476 - acc: 0.7184 - val_loss: 0.5291 - val_acc: 0.7353
Epoch 68/300
524038/524038 [==============================] - 324s - loss: 0.5486 - acc: 0.7174 - val_loss: 0.5261 - val_acc: 0.7383
Epoch 69/300
524038/524038 [==============================] - 324s - loss: 0.5476 

KeyboardInterrupt: 

Fit the complete train set using the number of epochs found above:

In [ ]:
#TODO do stuff with hist

In [11]:
model_path = "../output/models/for_validation_lstm_" + stamp + '.h5'
model.fit([q12_data, q21_data], double_labels, epochs=11, batch_size=batch_size, shuffle=True)
model.save(model_path)

RuntimeError: You must compile a model before training/testing. Use `model.compile(optimizer, loss)`.

Load the trained model and calculate logloss and accuarcy on the validation set:

In [ ]:
model = load_model(model_path)

predictions = model.predict([q1_val_data, q2_val_data], batch_size=batch_size, verbose=1)
predictions += model.predict([q2_val_data, q1_val_data], batch_size=batch_size, verbose=1)
predictions /= 2

loss = log_loss(val_y, predictions)
acc = accuracy_score(val_y, np.rint(predictions))

print "Validation scores of Lstm model\n LogLoss: {:.4f}\n Accuracy: {:.2f} ".format(loss, acc)

Create roc plot and save it:

In [ ]:
fpr, tpr, thresholds = roc_curve(val_y, predictions)
roc_auc = auc(fpr, tpr)

lw = 2
plt.plot([0, 1], [0, 1], linestyle='--', lw=lw, color='k', label='Luck')
plt.plot(fpr, tpr, color='darkorange', lw=lw, label='ROC curve (area = %0.2f)' % roc_auc)

plt.xlim([-0.05, 1.05])
plt.ylim([-0.05, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic example')
plt.legend(loc="lower right")
plt.savefig("../output/figures/lstm_roc_plot.png")
plt.show()